<a href="https://colab.research.google.com/github/mahalakshmijinadoss/pooling/blob/main/trac1hinbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[sentencepiece]==4.10.0 datasets --quiet


     |████████████████████████████████| 2.8 MB 5.4 MB/s 
     |████████████████████████████████| 311 kB 46.6 MB/s 
     |████████████████████████████████| 895 kB 43.0 MB/s 
     |████████████████████████████████| 67 kB 4.6 MB/s 
     |████████████████████████████████| 3.3 MB 35.8 MB/s 
     |████████████████████████████████| 596 kB 34.3 MB/s 
     |████████████████████████████████| 1.1 MB 46.3 MB/s 
     |████████████████████████████████| 133 kB 45.9 MB/s 
     |████████████████████████████████| 243 kB 47.5 MB/s 
     |████████████████████████████████| 1.1 MB 40.5 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 144 kB 46.8 MB/s 
     |████████████████████████████████| 271 kB 49.0 MB/s 


In [2]:
!python -c  "import transformers;print(transformers.__version__)"


4.10.0


In [3]:
!pip install -qq ipython-autotime

%load_ext autotime

time: 206 µs (started: 2022-02-09 15:56:30 +00:00)


In [4]:
import numpy as np
import torch
import random
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available

time: 2.13 s (started: 2022-02-09 15:56:51 +00:00)


In [5]:
def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf

        tf.random.set_seed(seed)

set_seed(1)

time: 3.33 s (started: 2022-02-09 15:56:55 +00:00)


In [6]:
import pandas as pd
import numpy as np


time: 1.17 ms (started: 2022-02-09 15:57:09 +00:00)


In [7]:
from transformers import DataCollatorWithPadding


time: 252 ms (started: 2022-02-09 15:57:11 +00:00)


In [8]:
!pip install wandb --quiet

import wandb
wandb.login()


     |████████████████████████████████| 1.7 MB 5.3 MB/s 
     |████████████████████████████████| 143 kB 47.4 MB/s 
     |████████████████████████████████| 180 kB 50.4 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

time: 15.5 s (started: 2022-02-09 15:57:14 +00:00)


In [9]:
%env WANDB_PROJECT=OGBV_task_trac1

env: WANDB_PROJECT=OGBV_task_trac1
time: 3.1 ms (started: 2022-02-09 15:57:58 +00:00)


In [10]:
!huggingface-cli login



        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/token.
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "rohanrajpal/bert-base-en-hi-codemix-cased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/832 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

time: 3.1 s (started: 2022-02-09 15:58:19 +00:00)


In [14]:
train= pd.read_csv('/content/agr_hi_train.csv', header=None,sep=',')

time: 102 ms (started: 2022-02-09 15:58:54 +00:00)


In [18]:
train.head()


,id,Text,Label
0,facebook_corpus_msr_401470,Mahmood Ghaznavi Aor ABdali ko bhol gaya ha tu...,OAG
1,facebook_corpus_msr_386695,Bhai 60sal pehle desh me kya tha pehle pta kro...,CAG
2,facebook_corpus_msr_373389,chutiya friday ko isliye releae krte kyoki wee...,CAG
3,facebook_corpus_msr_917635,जय मोदीराज,CAG
4,facebook_corpus_msr_382517,UPA walo ne bahot kuch kr diya tha desh k liye,CAG


time: 20.7 ms (started: 2022-02-09 16:00:54 +00:00)


In [19]:
train.columns =['id', 'Text', 'label']


time: 4.4 ms (started: 2022-02-09 16:01:06 +00:00)


In [20]:
train['label'].value_counts()

CAG    4869
OAG    4856
NAG    2275
Name: label, dtype: int64

time: 20.7 ms (started: 2022-02-09 16:01:17 +00:00)


In [21]:
train.loc[(train.label == 'OAG'), 'label'] = 1
train.loc[(train.label == 'CAG'), 'label'] = 1
train.loc[(train.label == 'NAG'), 'label'] = 0

time: 26.7 ms (started: 2022-02-09 16:01:45 +00:00)


In [25]:
dev= pd.read_csv('/content/agr_hi_train.csv',header=None)


time: 106 ms (started: 2022-02-09 16:02:26 +00:00)


In [27]:
dev.columns =['id', 'Text', 'label']


time: 4.58 ms (started: 2022-02-09 16:02:45 +00:00)


In [28]:
dev.loc[(dev.label == 'OAG'), 'label'] = 1
dev.loc[(dev.label == 'CAG'), 'label'] = 1
dev.loc[(dev.label == 'NAG'), 'label'] = 0

time: 31 ms (started: 2022-02-09 16:03:58 +00:00)


In [41]:
null_columns=train.columns[train.isnull().any()]
train[null_columns].isnull().sum()

Series([], dtype: float64)

time: 40.1 ms (started: 2022-02-09 16:11:11 +00:00)


In [43]:
null_columns=dev.columns[dev.isnull().any()]
dev[null_columns].isnull().sum()

Series([], dtype: float64)

time: 15.2 ms (started: 2022-02-09 16:11:28 +00:00)


In [40]:
train = train[train['Text'].notna()]


time: 16.5 ms (started: 2022-02-09 16:11:08 +00:00)


In [42]:
dev = dev[dev['Text'].notna()]


time: 6.46 ms (started: 2022-02-09 16:11:25 +00:00)


In [49]:
train_texts = train["Text"].tolist()
train_labels= train["label"].tolist()
dev_labels= dev["label"].tolist()
dev_texts = train["Text"].tolist()
num_labels = len(labels)

time: 9.71 ms (started: 2022-02-09 16:14:59 +00:00)


In [51]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

train_encodings = tokenizer(train_texts,truncation=True,)
dev_encodings = tokenizer(dev_texts,truncation=True)

time: 5.05 s (started: 2022-02-09 16:15:18 +00:00)


In [52]:
import torch

class TaskDatasetB(torch.utils.data.Dataset):

  def __init__(self,encodings,labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self,idx):
    item = {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

time: 10.6 ms (started: 2022-02-09 16:15:26 +00:00)


In [53]:
train_dataset = TaskDatasetB(train_encodings,train_labels)
dev_dataset = TaskDatasetB(dev_encodings,dev_labels)

time: 1.64 ms (started: 2022-02-09 16:15:29 +00:00)


In [54]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


time: 1.19 ms (started: 2022-02-09 16:15:31 +00:00)


In [55]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    'OGBV-gender-bert-hi-en-trac1',
    report_to = "wandb",
    run_name = "rr-bert-base-hi-en-3epoch",
    evaluation_strategy = 'epoch',
    save_strategy='epoch',
    num_train_epochs = 3,
    load_best_model_at_end = True,
    push_to_hub='OGBV-gender-bert-hi-en-trac1'
)

time: 91.4 ms (started: 2022-02-09 16:15:36 +00:00)


In [56]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels = num_labels,ignore_mismatched_sizes=True)


Downloading:   0%|          | 0.00/712M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at rohanrajpal/bert-base-en-hi-codemix-cased and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


time: 24 s (started: 2022-02-09 16:15:39 +00:00)


In [57]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score

def compute_metrics(eval_preds):

  logits,labels = eval_preds
  predictions  = np.argmax(logits,axis=-1)

  acc = accuracy_score(labels,predictions)
  precision = precision_score(labels,predictions,average='weighted')
  recall = recall_score(labels,predictions,average='weighted')
  f1 = f1_score(labels,predictions,average='weighted')
  
  return {
     'acc' : acc,
     'precision' : precision,
     'recall' : recall,
     'f1' : f1
  }

time: 7.69 ms (started: 2022-02-09 16:16:11 +00:00)


In [58]:
!apt-get install git-lfs


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-compute-11-0 cuda-nsight-compute-11-1
  cuda-nsight-systems-10-1 cuda-nsight-systems-

In [59]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = dev_dataset,

    data_collator = data_collator,
    compute_metrics = compute_metrics,
)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:1004: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/Maha/OGBV-gender-bert-hi-en-trac1 into local empty directory.


time: 19.5 s (started: 2022-02-09 16:16:22 +00:00)


In [60]:
trainer.train()


***** Running training *****
  Num examples = 11999
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4500
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: mahaj (use `wandb login --relogin` to force relogin)


Epoch,Training Loss,Validation Loss,Acc,Precision,Recall,F1
1,0.495000,0.486782,0.810484,0.656885,0.810484,0.725645
2,0.468900,0.536625,0.810484,0.656885,0.810484,0.725645
3,0.480300,0.509647,0.810484,0.656885,0.810484,0.725645


***** Running Evaluation *****
  Num examples = 11999
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to OGBV-gender-bert-hi-en-trac1/checkpoint-1500
Configuration saved in OGBV-gender-bert-hi-en-trac1/checkpoint-1500/config.json
Model weights saved in OGBV-gender-bert-hi-en-trac1/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 11999
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint 

TrainOutput(global_step=4500, training_loss=0.4861861131456163, metrics={'train_runtime': 3441.259, 'train_samples_per_second': 10.46, 'train_steps_per_second': 1.308, 'total_flos': 3027386122410180.0, 'train_loss': 0.4861861131456163, 'epoch': 3.0})

time: 57min 21s (started: 2022-02-09 16:16:45 +00:00)


In [61]:
trainer.evaluate()


***** Running Evaluation *****
  Num examples = 11999
  Batch size = 8


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'epoch': 3.0,
 'eval_acc': 0.8104842070172514,
 'eval_f1': 0.7256452691256463,
 'eval_loss': 0.4867819845676422,
 'eval_precision': 0.6568846498243829,
 'eval_recall': 0.8104842070172514,
 'eval_runtime': 275.4607,
 'eval_samples_per_second': 43.56,
 'eval_steps_per_second': 5.445}

time: 4min 35s (started: 2022-02-09 17:14:12 +00:00)


In [62]:
import torch
torch.cuda.empty_cache()

time: 774 ms (started: 2022-02-09 17:19:03 +00:00)


In [63]:
model_path = 'OGBV-gender-bert-hi-en-trac1'


time: 1.71 ms (started: 2022-02-09 17:19:09 +00:00)


In [64]:
model.push_to_hub('OGBV-gender-bert-hi-en-trac1')


Configuration saved in OGBV-gender-bert-hi-en-trac1/config.json
Model weights saved in OGBV-gender-bert-hi-en-trac1/pytorch_model.bin


Upload file pytorch_model.bin:   0%|          | 3.38k/679M [00:00<?, ?B/s]

To https://huggingface.co/Maha/OGBV-gender-bert-hi-en-trac1
   e7be5e9..8ab7b8f  main -> main



'https://huggingface.co/Maha/OGBV-gender-bert-hi-en-trac1/commit/8ab7b8f32d11d975a6bf6913ca81d098ce46d378'

time: 11min 37s (started: 2022-02-09 17:19:12 +00:00)


In [65]:
tokenizer.push_to_hub('OGBV-gender-bert-hi-en-trac1')


tokenizer config file saved in OGBV-gender-bert-hi-en-trac1/tokenizer_config.json
Special tokens file saved in OGBV-gender-bert-hi-en-trac1/special_tokens_map.json
To https://huggingface.co/Maha/OGBV-gender-bert-hi-en-trac1
   8ab7b8f..40a5f10  main -> main



'https://huggingface.co/Maha/OGBV-gender-bert-hi-en-trac1/commit/40a5f104c5cb9b27c80f738639f4ba6e4efb23da'

time: 6.41 s (started: 2022-02-09 17:30:54 +00:00)
